In [1]:
#You can easily compare the speed and memory usage of swin and Nat.
import torch
import torch.nn as nn
import torch.nn.functional as F
from timm.models.layers import DropPath, trunc_normal_,to_2tuple

import cupy
from collections import namedtuple
from string import Template

In [2]:
kernel_loop = '''
#define CUDA_KERNEL_LOOP(i, n)                        \
  for (int i = blockIdx.x * blockDim.x + threadIdx.x; \
      i < (n);                                       \
      i += blockDim.x * gridDim.x)
'''

nh_attn_forward_q_k = kernel_loop + '''
extern "C"
__global__ void nh_attn_forward_q_k(const ${Dtype}* query, const ${Dtype}* key, const ${Dtype}* bias, ${Dtype}* attn) {
    CUDA_KERNEL_LOOP(index, ${nthreads}) {
        const int b = index / ${num_heads} / ${height} / ${width} / ${window_seq_length};
        const int n_h = (index / ${height} / ${width}/ ${window_seq_length}) % ${num_heads};
        const int h = (index / ${width}/ ${window_seq_length}) % ${height};
        const int w = (index / ${window_seq_length}) % ${width};
        const int k = index % ${window_seq_length};
        
        const int kh = (k / ${window_size}) % ${window_size};
        const int kw = (k % ${window_size});
        
        int ph = ${shift_size};
        int pw = ${shift_size};
        int nh = h - ${shift_size};
        int nw = w - ${shift_size};
        
        if (nh < 0){
            nh = 0;
            ph = ${center_pos} - h;
        }
        else if (h + ${shift_size} >= ${height}){
            nh = ${height} - ${window_size};
            ph = ${height} - h - 1;
        }
        
        if (nw < 0){
            nw = 0;
            pw = ${center_pos} - w;
        }
        else if (w + ${shift_size} >= ${width}){
            nw = ${width} - ${window_size};
            pw = ${width} - w - 1;
        }
        
        const int q_idx = (((b * ${num_heads} + n_h) * ${height} + h) * ${width} + w) * ${channels};
        const int k_idx = (((b * ${num_heads} + n_h) * ${height} + (kh+nh)) * ${width} + (kw+nw)) * ${channels};
        const int b_idx = (n_h * ${bias_size} + (ph+kh)) * ${bias_size} + (pw+kw);
        
        if (h < ${height} && w < ${width} && n_h < ${num_heads}  && kh < ${window_size} && kw < ${window_size}){
            ${Dtype} update_value = 0;
            #pragma unroll
            for (int d=0; d < ${channels}; ++d){
                update_value += query[q_idx+d] * key[k_idx+d];
            }
            update_value += bias[b_idx];
            attn[index] = update_value;
        }
    }
}
'''

nh_attn_backward_query = kernel_loop + '''
extern "C"
__global__ void nh_attn_backward_query(const ${Dtype}* const key, const ${Dtype}* const d_attn, ${Dtype}* const d_query) {
    CUDA_KERNEL_LOOP(index, ${nthreads}) {
        const int b = index / ${num_heads} / ${height} / ${width} / ${channels};
        const int n_h = (index / ${height} / ${width}/ ${channels}) % ${num_heads};
        const int h = (index / ${width}/ ${channels}) % ${height};
        const int w = (index / ${channels}) % ${width};
        const int c = index % ${channels};
        
        int nh = max(h - ${shift_size}, 0) + (h + ${shift_size} >= ${height}) * (${height} - h - ${shift_size} - 1);
        int nw = max(w - ${shift_size}, 0) + (w + ${shift_size} >= ${width}) * (${width} - w - ${shift_size} - 1);
        
        const int a_idx = (((b * ${num_heads} + n_h) * ${height} + h) * ${width} + w)*${window_seq_length};
        
        if (h < ${height} && w < ${width} && n_h < ${num_heads}){
            ${Dtype} update_value = 0;
            #pragma unroll
            for (int kh=0, xh=nh; kh < ${window_size}; ++kh, ++xh){
                #pragma unroll
                for (int kw=0, xw=nw; kw < ${window_size}; ++kw, ++xw){
                     const int k_idx = ((((b * ${num_heads} + n_h) * ${height} + xh) * ${width} + xw) * ${channels} + c);
                     update_value += d_attn[a_idx+(kh*${window_size}+kw)] * key[k_idx];
                }
            }
            d_query[index] = update_value;
        }
    }
}
'''

nh_attn_backward_key = kernel_loop + '''
extern "C"
__global__ void nh_attn_backward_key(const ${Dtype}* const query, const ${Dtype}* const d_attn, ${Dtype}* const d_key) {
    CUDA_KERNEL_LOOP(index, ${nthreads}) {
        const int b = index / ${num_heads} / ${height} / ${width} / ${channels};
        const int n_h = (index / ${height} / ${width}/ ${channels}) % ${num_heads};
        const int h = (index / ${width}/ ${channels}) % ${height};
        const int w = (index / ${channels}) % ${width};
        const int c = index % ${channels};
        
        int nh = max(h - ${shift_size}, 0) + (h + ${shift_size} >= ${height}) * (${height} - h - ${shift_size} - 1);
        int nw = max(w - ${shift_size}, 0) + (w + ${shift_size} >= ${width}) * (${width} - w - ${shift_size} - 1);
        
        const int a_idx = (((b * ${num_heads} + n_h) * ${height} + h) * ${width} + w) * ${window_seq_length};
        
        if (h < ${height} && w < ${width} && n_h < ${num_heads}){
            ${Dtype} update_value = 0;
            #pragma unroll
            for (int kh=0, xh=nh; kh < ${window_size}; ++kh, ++xh){
                #pragma unroll
                for (int kw=0, xw=nw; kw < ${window_size}; ++kw, ++xw){
                    const int k_idx = ((((b * ${num_heads} + n_h) * ${height} + xh) * ${width} + xw) * ${channels} + c);
                    d_key[k_idx] += query[index] * d_attn[a_idx+(kh*${window_size}+kw)];
                }
            }
        }
    }
}
'''

nh_attn_backward_bias = kernel_loop + '''
extern "C"
__global__ void nh_attn_backward_bias(const ${Dtype}* const d_attn, ${Dtype}* const d_bias) {
      CUDA_KERNEL_LOOP(index, ${nthreads}) {
        const int n_h = (index / ${height} / ${width}/ ${window_seq_length}) % ${num_heads};
        const int h = (index / ${width}/ ${window_seq_length}) % ${height};
        const int w = (index / ${window_seq_length}) % ${width};
        const int k = index % ${window_seq_length};
        
        const int kh = (k / ${window_size}) % ${window_size};
        const int kw = (k % ${window_size});
        
        int ph = ${shift_size};
        int pw = ${shift_size};
        
        if (h < ${shift_size}){
            ph = ${center_pos} - h;
        }
        else if (h + ${shift_size} >= ${height}){
            ph = ${height} - h - 1;
        }
        
        if (w < ${shift_size}){
            pw = ${center_pos} - w;
        }
        else if (w + ${shift_size} >= ${width}){
            pw = ${width} - w - 1;
        }
        
        const int b_idx = (n_h * ${bias_size} + (ph+kh)) * ${bias_size} + (pw+kw);
        
        if (h < ${height} && w < ${width} && n_h < ${num_heads}  && kh < ${window_size} && kw < ${window_size}){
            d_bias[b_idx] += d_attn[index];
        }
    }
}

'''
nh_attn_forward_attn_v = kernel_loop + '''
extern "C"
__global__ void nh_attn_forward_attn_v(const ${Dtype}* attn, const ${Dtype}* value, ${Dtype}* out) {
    CUDA_KERNEL_LOOP(index, ${nthreads}) {
        const int b = index / ${num_heads} / ${height} / ${width} / ${channels};
        const int n_h = (index / ${height} / ${width}/ ${channels}) % ${num_heads};
        const int h = (index / ${width}/ ${channels}) % ${height};
        const int w = (index / ${channels}) % ${width};
        const int c = index % ${channels};
        
        int nh = max(h - ${shift_size}, 0) + (h + ${shift_size} >= ${height}) * (${height} - h - ${shift_size} - 1);
        int nw = max(w - ${shift_size}, 0) + (w + ${shift_size} >= ${width}) * (${width} - w - ${shift_size} - 1);
        
        const int a_idx = (((b * ${num_heads} + n_h) * ${height} + h) * ${width} + w)*${window_seq_length};
        
        if (h < ${height} && w < ${width} && n_h < ${num_heads}){
            ${Dtype} update_value = 0;
            #pragma unroll
            for (int kh=0, xh=nh; kh < ${window_size}; ++kh, ++xh){
                #pragma unroll
                for (int kw=0, xw=nw; kw < ${window_size}; ++kw, ++xw){
                     const int v_idx = ((((b * ${num_heads} + n_h) * ${height} + xh) * ${width} + xw) * ${channels} + c);
                     update_value += attn[a_idx+(kh*${window_size}+kw)] * value[v_idx];
                }
            }
            out[index] = update_value;
        }
    }
}

'''
nh_attn_backward_attn = kernel_loop + '''
extern "C"
__global__ void nh_attn_backward_attn(const ${Dtype}* const value, const ${Dtype}* const d_out, ${Dtype}* const d_attn) {
    CUDA_KERNEL_LOOP(index, ${nthreads}) {
        const int b = index / ${num_heads} / ${height} / ${width} / ${window_seq_length};
        const int n_h = (index / ${height} / ${width}/ ${window_seq_length}) % ${num_heads};
        const int h = (index / ${width}/ ${window_seq_length}) % ${height};
        const int w = (index / ${window_seq_length}) % ${width};
        const int k = index % ${window_seq_length};
        
        const int kh = (k / ${window_size}) % ${window_size};
        const int kw = (k % ${window_size});
        
        int nh = max(h - ${shift_size}, 0) + (h + ${shift_size} >= ${height}) * (${height} - h - ${shift_size} - 1);
        int nw = max(w - ${shift_size}, 0) + (w + ${shift_size} >= ${width}) * (${width} - w - ${shift_size} - 1);
        
        const int o_idx = (((b * ${num_heads} + n_h) * ${height} + h) * ${width} + w) * ${channels};
        const int v_idx = (((b * ${num_heads} + n_h) * ${height} + (nh+kh))* ${width} + (nw+kw)) * ${channels};
        
        if (h < ${height} && w < ${width} && n_h < ${num_heads}  && kh < ${window_size} && kw < ${window_size}){
            ${Dtype} update_value = 0;
            #pragma unroll
            for (int d=0; d < ${channels}; ++d){
                update_value += d_out[o_idx+d] * value[v_idx+d];
            }
            d_attn[index] = update_value;
        }
    }
}
'''

nh_attn_backward_value = kernel_loop + '''
extern "C"
__global__ void nh_attn_backward_value(const ${Dtype}* const attn, const ${Dtype}* const d_out, ${Dtype}* const d_value) {
    CUDA_KERNEL_LOOP(index, ${nthreads}) {
        const int b = index / ${num_heads} / ${height} / ${width} / ${channels};
        const int n_h = (index / ${height} / ${width}/ ${channels}) % ${num_heads};
        const int h = (index / ${width}/ ${channels}) % ${height};
        const int w = (index / ${channels}) % ${width};
        const int c = index % ${channels};
        
        int nh = max(h - ${shift_size}, 0) + (h + ${shift_size} >= ${height}) * (${height} - h - ${shift_size} - 1);
        int nw = max(w - ${shift_size}, 0) + (w + ${shift_size} >= ${width}) * (${width} - w - ${shift_size} - 1);
        
        const int a_idx = (((b * ${num_heads} + n_h) * ${height} + h) * ${width} + w) * ${window_seq_length};
        
        if (h < ${height} && w < ${width} && n_h < ${num_heads}){
            ${Dtype} update_value = 0;
            #pragma unroll
            for (int kh=0, xh=nh; kh < ${window_size}; ++kh, ++xh){
                #pragma unroll
                for (int kw=0, xw=nw; kw < ${window_size}; ++kw, ++xw){
                     const int v_idx = ((((b * ${num_heads} + n_h) * ${height} + xh) * ${width} + xw) * ${channels} + c);
                     d_value[v_idx] += attn[a_idx+(kh*${window_size}+kw)] * d_out[index];
                }
            }
        }
    }
}
'''

In [3]:
CUDA_NUM_THREADS = 1024
Stream = namedtuple('Stream', ['ptr'])

def GET_BLOCKS(N):
    return (N + CUDA_NUM_THREADS - 1) // CUDA_NUM_THREADS

def Dtype(t):
    if isinstance(t, torch.cuda.FloatTensor):
        return 'float'
    elif isinstance(t, torch.cuda.DoubleTensor):
        return 'double'
    
@cupy._util.memoize(for_each_device=True)
def load_kernel(kernel_name, code, **kwargs):
    code = Template(code).substitute(**kwargs)
    kernel_code = cupy.cuda.compile_with_cache(code)
    return kernel_code.get_function(kernel_name)

class nh_attn_q_k(torch.autograd.Function):
    @staticmethod
    def forward(ctx, query, key, bias, window_size):
        assert query.dim() == 5 and query.is_cuda
        assert key.dim() == 5 and key.is_cuda
        assert bias.dim() == 3 and bias.is_cuda
        
        batch_size, num_heads, height, width ,channels = query.size()
        attn = query.new(batch_size, num_heads, height, width,window_size**2)
        
        with torch.cuda.device_of(query):
            n = attn.numel()
            opt = dict(Dtype=Dtype(query), nthreads=n,
                       batch=batch_size, num_heads=num_heads, height=height, width=width, channels=channels,
                       window_size=window_size, window_seq_length=window_size**2, bias_size=(2*window_size-1),
                       center_pos=window_size-1, shift_size=window_size//2)
            f = load_kernel('nh_attn_forward_q_k', nh_attn_forward_q_k, **opt)
            
            f(block=(CUDA_NUM_THREADS,1,1),
              grid=(GET_BLOCKS(n),1,1),
              args=[query.data_ptr(), key.data_ptr(), bias.data_ptr(), attn.data_ptr()],
              stream=Stream(ptr=torch.cuda.current_stream().cuda_stream))
            
        ctx.save_for_backward(query, key, bias)
        ctx.window_size = window_size
        return attn
    
    @staticmethod
    def backward(ctx, d_attn):
        assert d_attn.is_cuda
        
        query, key, bias = ctx.saved_tensors
        window_size = ctx.window_size
        
        batch_size, num_heads, height, width ,channels = query.size()
        d_query, d_key, d_bias = None, None, None
        
        with torch.cuda.device_of(d_attn):
            if ctx.needs_input_grad[0]:
                d_query = query.new(query.size())
                n = d_query.numel()
                opt = dict(Dtype=Dtype(d_attn), nthreads=n,
                           batch=batch_size, num_heads=num_heads, height=height, width=width, channels=channels,
                           window_size=window_size, window_seq_length=window_size**2, bias_size=(2*window_size-1),
                           center_pos=window_size-1, shift_size=window_size//2)
                
                f = load_kernel('nh_attn_backward_query',nh_attn_backward_query, **opt)
                f(block=(CUDA_NUM_THREADS,1,1),
                  grid=(GET_BLOCKS(n),1,1),
                  args=[key.data_ptr(), d_attn.data_ptr(), d_query.data_ptr()],
                  stream=Stream(ptr=torch.cuda.current_stream().cuda_stream))
                
            if ctx.needs_input_grad[1]:
                d_key = key.new(key.size())
                n = d_key.numel()
                opt = dict(Dtype=Dtype(d_attn), nthreads=n,
                           batch=batch_size, num_heads=num_heads, height=height, width=width, channels=channels,
                           window_size=window_size, window_seq_length=window_size**2, bias_size=(2*window_size-1),
                           center_pos=window_size-1, shift_size=window_size//2)
                
                f = load_kernel('nh_attn_backward_key',nh_attn_backward_key, **opt)
                f(block=(CUDA_NUM_THREADS,1,1),
                  grid=(GET_BLOCKS(n),1,1),
                  args=[query.data_ptr(), d_attn.data_ptr(), d_key.data_ptr()],
                  stream=Stream(ptr=torch.cuda.current_stream().cuda_stream))
                
            if ctx.needs_input_grad[2]:
                d_bias = bias.new(bias.size())
                n = d_attn.numel()
                opt = dict(Dtype=Dtype(d_attn), nthreads=n,
                           batch=batch_size, num_heads=num_heads, height=height, width=width, channels=channels,
                           window_size=window_size, window_seq_length=window_size**2, bias_size=(2*window_size-1),
                           center_pos=window_size-1, shift_size=window_size//2)
                
                f = load_kernel('nh_attn_backward_bias',nh_attn_backward_bias, **opt)
                f(block=(CUDA_NUM_THREADS,1,1),
                  grid=(GET_BLOCKS(n),1,1),
                  args=[d_attn.data_ptr(), d_bias.data_ptr()],
                  stream=Stream(ptr=torch.cuda.current_stream().cuda_stream))
                
        return d_query, d_key, d_bias, None
    
class nh_attn_attn_v(torch.autograd.Function):
    @staticmethod
    def forward(ctx,attn,value,window_size):
        assert attn.dim() == 5 and attn.is_cuda
        assert value.dim() == 5 and value.is_cuda
        
        batch_size, num_heads, height, width ,channels = value.size()
        out = value.new(batch_size, num_heads, height, width, channels)
        
        with torch.cuda.device_of(attn):
            n = out.numel()
            opt = dict(Dtype=Dtype(attn), nthreads=n,
                       batch=batch_size, num_heads=num_heads, height=height, width=width, channels=channels,
                       window_size=window_size, window_seq_length=window_size**2, bias_size=(2*window_size-1),
                       center_pos=window_size-1, shift_size=window_size//2)
            
            f = load_kernel('nh_attn_forward_attn_v', nh_attn_forward_attn_v, **opt)
            f(block=(CUDA_NUM_THREADS,1,1),
              grid=(GET_BLOCKS(n),1,1),
              args=[attn.data_ptr(), value.data_ptr(), out.data_ptr()],
              stream=Stream(ptr=torch.cuda.current_stream().cuda_stream))
            
        ctx.save_for_backward(attn, value)
        ctx.window_size = window_size
        return out
    
    @staticmethod
    def backward(ctx, d_out):
        assert d_out.is_cuda
        
        attn, value = ctx.saved_tensors
        window_size = ctx.window_size
        
        batch_size, num_heads, height, width ,channels = value.size()
        d_attn, d_value = None, None
        
        with torch.cuda.device_of(d_out):
            if ctx.needs_input_grad[0]:
                d_attn = attn.new(attn.size())
                n = d_attn.numel()
                opt = dict(Dtype=Dtype(d_out), nthreads=n,
                           batch=batch_size, num_heads=num_heads, height=height, width=width, channels=channels,
                           window_size=window_size, window_seq_length=window_size**2, bias_size=(2*window_size-1),
                           center_pos=window_size-1, shift_size=window_size//2)
                
                f = load_kernel('nh_attn_backward_attn',nh_attn_backward_attn, **opt)
                f(block=(CUDA_NUM_THREADS,1,1),
                  grid=(GET_BLOCKS(n),1,1),
                  args=[value.data_ptr(), d_out.data_ptr(), d_attn.data_ptr()],
                  stream=Stream(ptr=torch.cuda.current_stream().cuda_stream))
                
            if ctx.needs_input_grad[1]:
                d_value = value.new(value.size())
                n = d_value.numel()
                opt = dict(Dtype=Dtype(d_out), nthreads=n,
                           batch=batch_size, num_heads=num_heads, height=height, width=width, channels=channels,
                           window_size=window_size, window_seq_length=window_size**2, bias_size=(2*window_size-1),
                           center_pos=window_size-1, shift_size=window_size//2)
                
                f = load_kernel('nh_attn_backward_value',nh_attn_backward_value, **opt)
                f(block=(CUDA_NUM_THREADS,1,1),
                  grid=(GET_BLOCKS(n),1,1),
                  args=[attn.data_ptr(), d_out.data_ptr(), d_value.data_ptr()],
                  stream=Stream(ptr=torch.cuda.current_stream().cuda_stream))
                
        return d_attn, d_value,None
    
class NeighborhoodAttention(nn.Module):
    def __init__(self,dim, num_heads,window_size=7, qkv_bias=True, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        assert window_size%2 == 1,'windowsize must be odd.'
        self.dim = dim
        self.num_heads = num_heads
        self.window_size = window_size
        self.head_dim = dim // num_heads
        self.scale = qk_scale or self.head_dim ** -0.5
        
        self.qkv = nn.Conv2d(dim,dim*3,1, bias=qkv_bias)
        self.proj = nn.Conv2d(dim, dim, 1)
        self.proj_drop = nn.Dropout(proj_drop)
        self.attn_drop = nn.Dropout(attn_drop)
        self.relative_bias = nn.Parameter(torch.zeros(num_heads,(2*self.window_size-1),(2*self.window_size-1)))
        
        trunc_normal_(self.relative_bias, std=.02)
        
    def forward(self, x):
        x = self.nh_attention(x)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x
    
    def nh_attention(self,x):
        B,C,H,W = x.shape
        assert H >= self.window_size and W >= self.window_size,'input size must not be smaller than window size'
        qkv = self.qkv(x).view(B, 3,self.num_heads,self.head_dim,H,W).permute(1,0,2,4,5,3) # B,nh,H,W,nc
        q, k, v = qkv[0], qkv[1] ,qkv[2]
        attn = self.nh_attn(q,k,mode='q_k')
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)
        out = self.nh_attn(attn,v,mode='attn_v')
        out = out.permute(0,1,4,2,3).contiguous().view(B,C,H,W)
        return out
    
    def nh_attn(self,input_1, input_2,mode='q_k'):
        if input_1.is_cuda and input_2.is_cuda and self.relative_bias.is_cuda:
            if mode.lower() == 'q_k':
                attn = nh_attn_q_k.apply(input_1, input_2,self.relative_bias,self.window_size)
                return attn
            elif mode.lower() == 'attn_v':
                out = nh_attn_attn_v.apply(input_1, input_2,self.window_size)
                return out
            else:
                raise NotImplementedError
        else:
            raise NotImplementedError
            
class Channel_Layernorm(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.ln = nn.LayerNorm(dim)
        
    def forward(self, x):
        x = x.permute(0, 2, 3, 1)
        x = self.ln(x)
        x = x.permute(0, 3, 1, 2)
        return x
    
class Mlp_conv(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Conv2d(in_features, hidden_features,1)
        self.act = act_layer()
        self.fc2 = nn.Conv2d(hidden_features, out_features,1)
        self.drop = nn.Dropout(drop)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x
    
class NATLayer(nn.Module):
    def __init__(self, dim, num_heads,window_size=7,
                 mlp_ratio=4., qkv_bias=True, qk_scale=None, drop=0., attn_drop=0., drop_path=0.,
                 act_layer=nn.GELU, norm_layer=Channel_Layernorm, layer_scale=None):
        super().__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.mlp_ratio = mlp_ratio
        
        self.norm1 = norm_layer(dim)
        self.norm2 = norm_layer(dim)
        self.attn = NeighborhoodAttention(dim, num_heads,window_size,qkv_bias, qk_scale, attn_drop, drop)
        self.mlp = Mlp_conv(in_features=dim, hidden_features=int(dim * mlp_ratio), act_layer=act_layer, drop=drop)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        
    def forward(self, x):
        x = x + self.drop_path(self.attn(self.norm1(x)))
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x

In [4]:
#https://github.com/microsoft/Swin-Transformer/blob/main/models/swin_transformer.py
class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)
    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x
    
def window_partition(x, window_size):
    B, H, W, C = x.shape
    x = x.view(B, H // window_size, window_size, W // window_size, window_size, C)
    windows = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(-1, window_size, window_size, C)
    return windows

def window_reverse(windows, window_size, H, W):
    B = int(windows.shape[0] / (H * W / window_size / window_size))
    x = windows.view(B, H // window_size, W // window_size, window_size, window_size, -1)
    x = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(B, H, W, -1)
    return x

class WindowAttention(nn.Module):
    def __init__(self, dim, window_size, num_heads, qkv_bias=True, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.dim = dim
        self.window_size = window_size  # Wh, Ww
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5
        # define a parameter table of relative position bias
        self.relative_position_bias_table = nn.Parameter(
            torch.zeros((2 * window_size[0] - 1) * (2 * window_size[1] - 1), num_heads))  # 2*Wh-1 * 2*Ww-1, nH
        # get pair-wise relative position index for each token inside the window
        coords_h = torch.arange(self.window_size[0])
        coords_w = torch.arange(self.window_size[1])
        coords = torch.stack(torch.meshgrid([coords_h, coords_w]))  # 2, Wh, Ww
        coords_flatten = torch.flatten(coords, 1)  # 2, Wh*Ww
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]  # 2, Wh*Ww, Wh*Ww
        relative_coords = relative_coords.permute(1, 2, 0).contiguous()  # Wh*Ww, Wh*Ww, 2
        relative_coords[:, :, 0] += self.window_size[0] - 1  # shift to start from 0
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 0] *= 2 * self.window_size[1] - 1
        relative_position_index = relative_coords.sum(-1)  # Wh*Ww, Wh*Ww
        self.register_buffer("relative_position_index", relative_position_index)
        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)
        trunc_normal_(self.relative_position_bias_table, std=.02)
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, x, mask=None):
        B_, N, C = x.shape
        qkv = self.qkv(x).reshape(B_, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]  # make torchscript happy (cannot use tensor as tuple)
        q = q * self.scale
        attn = (q @ k.transpose(-2, -1))
        relative_position_bias = self.relative_position_bias_table[self.relative_position_index.view(-1)].view(
            self.window_size[0] * self.window_size[1], self.window_size[0] * self.window_size[1], -1)  # Wh*Ww,Wh*Ww,nH
        relative_position_bias = relative_position_bias.permute(2, 0, 1).contiguous()  # nH, Wh*Ww, Wh*Ww
        attn = attn + relative_position_bias.unsqueeze(0)
        if mask is not None:
            nW = mask.shape[0]
            attn = attn.view(B_ // nW, nW, self.num_heads, N, N) + mask.unsqueeze(1).unsqueeze(0)
            attn = attn.view(-1, self.num_heads, N, N)
            attn = self.softmax(attn)
        else:
            attn = self.softmax(attn)
        attn = self.attn_drop(attn)
        x = (attn @ v).transpose(1, 2).reshape(B_, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

class SwinTransformerBlock(nn.Module):
    def __init__(self, dim, input_resolution, num_heads, window_size=7, shift_size=0,
                 mlp_ratio=4., qkv_bias=True, qk_scale=None, drop=0., attn_drop=0., drop_path=0.,
                 act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.dim = dim
        self.input_resolution = input_resolution
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        self.mlp_ratio = mlp_ratio
        if min(self.input_resolution) <= self.window_size:
            # if window size is larger than input resolution, we don't partition windows
            self.shift_size = 0
            self.window_size = min(self.input_resolution)
        assert 0 <= self.shift_size < self.window_size, "shift_size must in 0-window_size"
        self.norm1 = norm_layer(dim)
        self.attn = WindowAttention(
            dim, window_size=to_2tuple(self.window_size), num_heads=num_heads,
            qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)
        if self.shift_size > 0:
            # calculate attention mask for SW-MSA
            H, W = self.input_resolution
            img_mask = torch.zeros((1, H, W, 1))  # 1 H W 1
            h_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            w_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            cnt = 0
            for h in h_slices:
                for w in w_slices:
                    img_mask[:, h, w, :] = cnt
                    cnt += 1
            mask_windows = window_partition(img_mask, self.window_size)  # nW, window_size, window_size, 1
            mask_windows = mask_windows.view(-1, self.window_size * self.window_size)
            attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
            attn_mask = attn_mask.masked_fill(attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))
        else:
            attn_mask = None
        self.register_buffer("attn_mask", attn_mask)
        
    def forward(self, x):
        H, W = self.input_resolution
        B, L, C = x.shape
        assert L == H * W, "input feature has wrong size"
        shortcut = x
        x = self.norm1(x)
        x = x.view(B, H, W, C)
        # cyclic shift
        if self.shift_size > 0:
            shifted_x = torch.roll(x, shifts=(-self.shift_size, -self.shift_size), dims=(1, 2))
        else:
            shifted_x = x
        # partition windows
        x_windows = window_partition(shifted_x, self.window_size)  # nW*B, window_size, window_size, C
        x_windows = x_windows.view(-1, self.window_size * self.window_size, C)  # nW*B, window_size*window_size, C
        # W-MSA/SW-MSA
        attn_windows = self.attn(x_windows, mask=self.attn_mask)  # nW*B, window_size*window_size, C
        # merge windows
        attn_windows = attn_windows.view(-1, self.window_size, self.window_size, C)
        shifted_x = window_reverse(attn_windows, self.window_size, H, W)  # B H' W' C
        # reverse cyclic shift
        if self.shift_size > 0:
            x = torch.roll(shifted_x, shifts=(self.shift_size, self.shift_size), dims=(1, 2))
        else:
            x = shifted_x
        x = x.view(B, H * W, C)
        # FFN
        x = shortcut + self.drop_path(x)
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x

In [5]:
def cuda_memory_test(model,img):
    print("torch.cuda.memory_reserved: %fGB"%(torch.cuda.memory_reserved(0)/1024/1024/1024))
    print("torch.cuda.max_memory_reserved: %fGB"%(torch.cuda.max_memory_reserved(0)/1024/1024/1024))
    model(img)
    print("torch.cuda.memory_reserved: %fGB"%(torch.cuda.memory_reserved(0)/1024/1024/1024))
    print("torch.cuda.max_memory_reserved: %fGB"%(torch.cuda.max_memory_reserved(0)/1024/1024/1024))
    
#https://towardsdatascience.com/the-correct-way-to-measure-inference-time-of-deep-neural-networks-304a54e5187f
def cuda_speed_test(model,img,repetitions):
    total_time = 0
    with torch.no_grad():
        for rep in range(repetitions):
            starter, ender = torch.cuda.Event(enable_timing=True),torch.cuda.Event(enable_timing=True)
            starter.record()
            _ = model(img)
            ender.record()
            torch.cuda.synchronize()
            curr_time = starter.elapsed_time(ender)/1000
            total_time += curr_time
    Throughput = (repetitions*batch_size)/total_time
    print(f'Final Throughput:{Throughput}')

In [6]:
batch_size = 32
img_size = 112
channel = 64
num_head = 4
window_size = 7
repetitions = 1000
test_model = 'nat' #'nat' or 'swin'
test_type = 'memory' # 'memory' or 'speed'

In [7]:
if test_model.lower() == 'nat':
    model = NATLayer(channel,num_head,window_size=window_size).cuda()
    img = torch.rand(batch_size,channel,img_size,img_size).cuda()
elif test_model.lower() == 'swin':
    model = SwinTransformerBlock(channel,(img_size,img_size),num_head,window_size=window_size).cuda()
    img = torch.rand(batch_size,img_size*img_size,channel).cuda()
else:
    raise NotImplementedError
    
if test_type.lower() == 'memory':
    cuda_memory_test(model,img)
elif test_type.lower() == 'speed':
    cuda_speed_test(model,img,repetitions)
else:
    raise NotImplementedError

C:\Users\qwopq\.conda\envs\tf\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Final Throughput:1376.9413621660358
